In [1]:
!pip install easyocr

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.9 MB 5.6 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 6.6 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 6.6 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 12.5 MB/s eta 0:00:00


In [1]:
import os
import cv2
import glob
import json
import torch
import easyocr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Parse data path

In [2]:
keyframes_dir = 'D:/AIChallenge/data_sotuyen1/keyframe'
all_keyframe_paths = dict()

# Duyệt qua tất cả các thư mục trong keyframes_dir
for part in sorted(os.listdir(keyframes_dir)):
    # Lấy tên phần chính (ví dụ: L01, L01_extra)
    data_part = part.split('_')[0] # Lấy phần L01 hoặc L01 từ L01_extra
    is_extra = '_extra' in part # Kiểm tra xem có phải là thư mục _extra hay không
    
    # Nếu là _extra, thì thêm vào key '_extra'
    data_key = f"{data_part}_extra" if is_extra else data_part
    
    # Khởi tạo từ điển con nếu chưa có
    if data_key not in all_keyframe_paths:
        all_keyframe_paths[data_key] = dict()

# Duyệt qua tất cả các phần dữ liệu
for data_key in sorted(all_keyframe_paths.keys()):
    # Xác định đường dẫn cho data_key, bao gồm cả phần _extra
    data_part_path = f"{keyframes_dir}/{data_key.replace('_extra', '')}{'_extra' if '_extra' in data_key else ''}"
    video_dirs = sorted(os.listdir(data_part_path))
    
    # Lấy video_id từ tên thư mục (ví dụ: từ L01_V031 lấy được V031)
    video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
    
    # Lưu keyframe paths cho từng video_id trong data_key
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_key][video_id] = keyframe_paths

# In thử kết quả để kiểm tra
for key, videos in all_keyframe_paths.items():
    print(f"Data Part: {key}, Number of videos: {len(videos)}")

Data Part: L01, Number of videos: 31
Data Part: L01_extra, Number of videos: 31
Data Part: L02, Number of videos: 31
Data Part: L02_extra, Number of videos: 31
Data Part: L03, Number of videos: 30
Data Part: L03_extra, Number of videos: 30
Data Part: L04, Number of videos: 30
Data Part: L04_extra, Number of videos: 30
Data Part: L05, Number of videos: 31
Data Part: L05_extra, Number of videos: 31
Data Part: L06, Number of videos: 31
Data Part: L06_extra, Number of videos: 31
Data Part: L07, Number of videos: 31
Data Part: L07_extra, Number of videos: 31
Data Part: L08, Number of videos: 30
Data Part: L08_extra, Number of videos: 30
Data Part: L09, Number of videos: 29
Data Part: L09_extra, Number of videos: 29
Data Part: L10, Number of videos: 29
Data Part: L10_extra, Number of videos: 29
Data Part: L11, Number of videos: 30
Data Part: L11_extra, Number of videos: 30
Data Part: L12, Number of videos: 30
Data Part: L12_extra, Number of videos: 30
Data Part: L13, Number of videos: 30
Dat

# Run inference

In [3]:
reader = easyocr.Reader(['vi'], gpu=True) # this needs to run only once to load the model into memory

d:\anaconda\envs\py310\lib\site-packages\easyocr\detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, m

In [8]:
bs = 16
save_dir = './ocr'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

keys = sorted(all_keyframe_paths.keys())
for key in tqdm(keys):
    video_keyframe_paths = all_keyframe_paths[key]
    video_ids = sorted(video_keyframe_paths.keys())

    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))

    for video_id in tqdm(video_ids):
        video_keyframe_path = video_keyframe_paths[video_id]
        video_ocr_results = []
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            image_paths = video_keyframe_path[i:i+bs]
            results = reader.readtext_batched(image_paths, batch_size=len(image_paths))
            for result in results:
                refined_result = []
                for item in result: 
                    if item[2] > 0.5:
                        refined_result.append(item)   
                refined_result = easyocr.utils.get_paragraph(refined_result)
                text_detected = [item[1] for item in refined_result]
                video_ocr_results.append(text_detected)

        with open(f'{save_dir}/{key}/{video_id}.json',"w", encoding='utf-8') as jsonfile:
            json.dump(video_ocr_results, jsonfile, ensure_ascii=False)

  0%|          | 0/48 [00:00<?, ?it/s]